In [ ]:
#Necessary Imports
!pip install ta
import numpy as np
import pandas as pd
import os
import pickle
import ta
from google.colab import drive
drive.mount('/content/drive')

#Data Source Location
os.chdir("/content/drive/My Drive/mining-on-stock")

Mounted at /content/drive


In [ ]:
#500 Stock dataset
with open('testing_set1.pkl', 'rb') as train_file:
  train_data = pickle.load(train_file)

In [ ]:
len(train_data[0])

889

In [ ]:
# Separate the features and the labels
temp_data = train_data.drop(['vwap (None)'], axis=1)
temp_data = temp_data.dropna(subset=['Label'])

X = temp_data.iloc[:, :-1]
y = temp_data.iloc[:, -1]

# Check for NaN values
# print(X.isnull().sum())
X = X.fillna(X.mean())
# print(X.isnull().sum())

labels = y
labels_encoded = np.zeros((len(labels), 3))
labels_encoded[labels == 'increase', 2] = 1
labels_encoded[labels == 'no big change', 1] = 1
labels_encoded[labels == 'decrease', 0] = 1
y_encoded = labels_encoded

In [ ]:
from sklearn.model_selection import train_test_split

#Fix Max value i.e normalize
X[np.isfinite(X) == False] = np.finfo('float64').max
X = np.clip(X, a_min=None, a_max=1e10)

# Split data into training set and validation set
# X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42) #test_size 0.5/0.2 (to be decided)
# print(len(X_train), len(X_val), len(y_train), len(y_val))

# X_train = X_train.dropna()
# print(len(X_train), len(X_val), len(y_train), len(y_val))

In [ ]:
print(temp_data.shape, X.shape, y_encoded.shape)

(1100500, 109) (1100500, 108) (1100500, 3)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

# Split data into training and validation sets
X = X.values
y = y_encoded
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X_train and X_val to represent past 100 trading records for each stock (TO BE VERIFIED) (USED RESHAPE TECHNIQUE) (TRY SLIDING WINDOW TECHNIQUE)
X_train = X_train.reshape(-1, 100, X_train.shape[1])
X_val = X_val.reshape(-1, 100, X_val.shape[1])

# Convert labels to 3-element vectors
# convert labels to integers
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_val = le.transform(y_val)
# y_train = keras.utils.to_categorical(y_train, num_classes=3)
# y_val = keras.utils.to_categorical(y_val, num_classes=3)

# Train Random Forest model
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train.reshape(-1, X_train.shape[2]), y_train.reshape(-1,))

# # Evaluate model on training and validation sets
# for name, X, y in [('Training', X_train, y_train), ('Validation', X_val, y_val)]:
#     y_pred = rf_model.predict(X.reshape(-1, X.shape[2])).reshape(-1, 3)
#     acc = accuracy_score(np.argmax(y, axis=1), np.argmax(y_pred, axis=1))
#     precision = precision_score(np.argmax(y, axis=1), np.argmax(y_pred, axis=1), pos_label=2)
#     pos_pred = np.sum(np.argmax(y_pred, axis=1) == 2) / y_pred.shape[0]
#     print(f'{name} accuracy: {acc:.4f}, precision: {precision:.4f}, percentage of positive predictions: {pos_pred:.4f}')


In [ ]:
X_train.reshape(-1, X_train.shape[2]).shape

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train.reshape(-1, X_train.shape[2]), y_train)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Define the parameter grid for the randomized search
param_dist = {'n_estimators': randint(100, 1000),
              'max_features': ['sqrt', 'log2'],
              'max_depth': [None] + list(randint(3, 20).rvs(5)),
              'min_samples_split': randint(2, 10)}

# Create a Random Forest classifier object
rf_model = RandomForestClassifier(random_state=42)

# Perform randomized search cross-validation
random_search = RandomizedSearchCV(estimator=rf_model,
                                   param_distributions=param_dist,
                                   cv=5,
                                   n_iter=2,
                                   n_jobs=-1,
                                   random_state=42)

# Fit the randomized search to the training data
random_search.fit(X_train.reshape(-1, X_train.shape[2]), y_train)

# Get the best estimator
best_estimator = random_search.best_estimator_

# Evaluate the performance of the best estimator on the validation data
accuracy = best_estimator.score(X_val.reshape(-1, X_val.shape[2]), y_val)
precision = precision_score(y_val, best_estimator.predict(X_val.reshape(-1, X_val.shape[2])))


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from itertools import product

# Define the parameter grid
param_grid = {'n_estimators': [100, 500, 1000],
              'max_features': ['sqrt', 'log2'],
              'max_depth': [None, 5, 10, 15],
              'min_samples_split': [2, 5, 8]}

# Generate all combinations of hyperparameters
hyperparams = list(product(*param_grid.values()))

# Create a list to store the models
models = []

# Train a model for each combination of hyperparameters
for params in hyperparams:
    rf_model = RandomForestClassifier(random_state=42, **dict(zip(param_grid.keys(), params)))
    scores = cross_val_score(rf_model, X_train.reshape(-1, X_train.shape[2]), y_train, cv=5)
    mean_score = scores.mean()
    models.append({'params': params, 'score': mean_score})
    break

# Find the model with the highest mean score
best_model = max(models, key=lambda x: x['score'])

# Train the best model on the full training set
best_rf_model = RandomForestClassifier(random_state=42, **dict(zip(param_grid.keys(), best_model['params'])))
best_rf_model.fit(X_train.reshape(-1, X_train.shape[2]), y_train)

# Evaluate the performance of the best model on the validation set
accuracy = best_rf_model.score(X_val.reshape(-1, X_val.shape[2]), y_val)
precision = precision_score(y_val, best_rf_model.predict(X_val.reshape(-1, X_val.shape[2])))


In [ ]:
# Define the Random Forest classifier with hyperparameters
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_split=5, max_features='sqrt', random_state=42)

# Fit the model with cross-validation
cv_results = cross_validate(rf_model, 
                             X_train.reshape(-1, X_train.shape[2]), 
                             y_train, 
                             cv=2, 
                             scoring=('accuracy', 'precision'),
                             return_train_score=True)

# Print the mean accuracy and precision scores across the 5 folds
print("Accuracy (train): {:.3f}".format(cv_results['train_accuracy'].mean()))
print("Accuracy (test): {:.3f}".format(cv_results['test_accuracy'].mean()))
print("Precision (train): {:.3f}".format(cv_results['train_precision'].mean()))
print("Precision (test): {:.3f}".format(cv_results['test_precision'].mean()))

# Fit the model on the entire training set
rf_model.fit(X_train.reshape(-1, X_train.shape[2]), y_train)

# Evaluate the model on the validation set
accuracy = rf_model.score(X_val.reshape(-1, X_val.shape[2]), y_val)
precision = precision_score(y_val, rf_model.predict(X_val.reshape(-1, X_val.shape[2])))
print("Accuracy (validation): {:.3f}".format(accuracy))
print("Precision (validation): {:.3f}".format(precision))


KeyboardInterrupt: ignored

In [ ]:
joblib.dump(random_search, 'random_search_rf.pkl')

In [ ]:
for name, X, y in [('Training', X_train, y_train), ('Validation', X_val, y_val)]:
    y_pred = rf_model.predict(X.reshape(-1, X.shape[2])).reshape(-1, 3)
    acc = accuracy_score(np.argmax(y, axis=1), np.argmax(y_pred, axis=1))
    precision = precision_score(np.argmax(y, axis=1), np.argmax(y_pred, axis=1), average="weighted")
    pos_pred = np.sum(np.argmax(y_pred, axis=1) == 2) / y_pred.shape[0]
    print(f'{name} accuracy: {acc:.4f}, precision: {precision:.4f}, percentage of positive predictions: {pos_pred:.4f}')

Training accuracy: 1.0000, precision: 1.0000, percentage of positive predictions: 0.3328
Validation accuracy: 0.7482, precision: 0.7624, percentage of positive predictions: 0.2270


In [ ]:
for name, X, y in [('Training', X_train, y_train), ('Validation', X_val, y_val)]:
    y_pred = rf_model.predict(X.reshape(-1, X.shape[2])).reshape(-1, 3)
    acc = accuracy_score(np.argmax(y, axis=1), np.argmax(y_pred, axis=1))
    precision = precision_score(np.argmax(y, axis=1), np.argmax(y_pred, axis=1), average="weighted")
    pos_pred = np.sum(np.argmax(y_pred, axis=1) == 2) / y_pred.shape[0]
    print(f'{name} accuracy: {acc:.4f}, precision: {precision:.4f}, percentage of positive predictions: {pos_pred:.4f}')

Training accuracy: 1.0000, precision: 1.0000, percentage of positive predictions: 0.3338
Validation accuracy: 0.7482, precision: 0.7624, percentage of positive predictions: 0.3211


In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

# Load and split data into primary training set and validation set
# train_data = pd.read_csv("train_data.csv")
# X = train_data.iloc[:, :-1]
# y = train_data.iloc[:, -1]
# labels = y
# labels_encoded = np.zeros((len(labels), 3))
# labels_encoded[labels == 'increase', 2] = 1
# labels_encoded[labels == 'no big change', 1] = 1
# labels_encoded[labels == 'decrease', 0] = 1
# y_encoded = labels_encoded

X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

# Initialize the neural network
input_size = X_train.shape[1]
hidden_size1 = 256
hidden_size2 = 128
hidden_size3 = 64
output_size = 3

net = Net(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Train the neural network
num_epochs = 20
batch_size = 128

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        batch_X = torch.Tensor(X_train[i:i+batch_size].values)
        batch_y = torch.Tensor(y_train[i:i+batch_size])
        
        # Forward pass
        outputs = net(batch_X)
        loss = criterion(outputs, torch.max(batch_y, 1)[1])
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Compute accuracy and precision on validation set
    with torch.no_grad():
        val_X = torch.Tensor(X_val.values)
        val_y = torch.Tensor(y_val)
        val_outputs = net(val_X)
        _, predicted = torch.max(val_outputs.data, 1)
        total = val_y.size(0)
        correct = (predicted == torch.max(val_y, 1)[1]).sum().item()
        accuracy = correct / total
        precision = precision_score(torch.max(val_y, 1)[1], predicted, average='macro')
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/20], Loss: nan, Accuracy: 0.3336, Precision: 0.1112


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [2/20], Loss: nan, Accuracy: 0.3336, Precision: 0.1112


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch [12/20], Loss: nan, Accuracy: 0.3336, Precision: 0.1112


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [13/20], Loss: nan, Accuracy: 0.3336, Precision: 0.1112


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [14/20], Loss: nan, Accuracy: 0.3336, Precision: 0.1112


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

one_hot_encoded = pd.get_dummies(temp_data["Label"])
y_ = np.array(one_hot_encoded)
y_encoded = list(y_)
print(len(X), len(y_encoded))

1100500 1100500


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, make_scorer
def result_metrics(y_train, y_train_pred, y_val, y_val_pred, cv_results):

  train_accuracy = accuracy_score(y_train, y_train_pred)
  train_precision = precision_score(y_train, y_train_pred, average='weighted')

  val_accuracy = accuracy_score(y_val, y_val_pred)
  val_precision = precision_score(y_val, y_val_pred, average='weighted')

  # calculate the percentage of positive predictions (to be decided)
  train_pct_pos = sum(y_train_pred == np.array([0,0,1])) / len(y_train_pred) * 100
  val_pct_pos = sum(y_val_pred == np.array([0,0,1])) / len(y_val_pred) * 100

  # print the results
  print("Training set accuracy:", train_accuracy)
  print("Training set precision:", train_precision)
  print("Validation set accuracy:", val_accuracy)
  print("Validation set precision:", val_precision)
  print("Percentage of positive predictions on training set:", train_pct_pos)
  print("Percentage of positive predictions on validation set:", val_pct_pos)
  print("Mean accuracy score from cross-validation:", cv_results['test_accuracy'].mean())
  print("Standard deviation of accuracy score from cross-validation:", cv_results['test_accuracy'].std())
  print("Mean precision score from cross-validation:", cv_results['test_precision'].mean())
  print("Standard deviation of precision score from cross-validation:", cv_results['test_precision'].std())

In [ ]:
import joblib
def save_model(model, model_name):
  # save the trained model object into pkl file
  joblib.dump(model, f'{model_name}.pkl')

In [ ]:
import torch
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8 * 8 * 32, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 8 * 8 * 32)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x


In [ ]:
#HistGradientBoostingClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

# define the model with optimized hyperparameters
hist_model = HistGradientBoostingClassifier(
    max_depth=10,
    max_iter=200,
    learning_rate=0.1,
    min_samples_leaf=50,
    max_leaf_nodes=5000,
    validation_fraction=0.1,
    n_iter_no_change=10,
    verbose=1,
    random_state=42,
    # loss='log_loss'
)

# define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score, average='weighted')}
hist_model = MultiOutputClassifier(hist_model, n_jobs=-1)

# compute the cross-validation scores
cv_results = cross_validate(hist_model, X_train, y_train, cv=5, scoring=scoring)

# train the model on the training sethist_model = MultiOutputClassifier(hist_model, n_jobs=-1)
hist_model.fit(X_train, y_train)d

# predict on the training set and compute the accuracy and precision scores
y_train_pred = hist_model.predict(X_train)

# predict on the validation set and compute the accuracy and precision scores
y_val_pred = hist_model.predict(X_val)

#Call to result_metrics
result_metrics(y_train, y_train_pred, y_val, y_val_pred, cv_results)

#save the model
save_model(hist_model, 'hist_model_cv')

Training set accuracy: 0.8244547932757837
Training set precision: 0.8805115950590375
Validation set accuracy: 0.7709541117673785
Validation set precision: 0.8247498540599081
Percentage of positive predictions on training set: [65.40731486 73.24068605 35.11063153]
Percentage of positive predictions on validation set: [65.05270332 73.44934121 35.31985461]
Mean accuracy score from cross-validation: 0.7692776010904134
Standard deviation of accuracy score from cross-validation: 0.0006599712915076703
Mean precision score from cross-validation: 0.8236228956345977
Standard deviation of precision score from cross-validation: 0.0006606699787418656


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define preprocessing pipeline
preprocessor = Pipeline([
    ('scaler', StandardScaler())
])

# Define logistic regression model with L2 regularization and cross-validation
lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=3000, random_state=42, class_weight='balanced')

# Define the final pipeline that combines preprocessing and logistic regression
lr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('lr', lr)
])

# Define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score, average='weighted')}

# Compute cross-validation scores
# cv_results = cross_validate(lr_model, X, y_encoded, cv=2, scoring=scoring)

# Fit the model to the training data
lr_model.fit(X_train, y_train)

# Predict on the training set and compute the accuracy and precision scores
y_train_pred = lr_model.predict(X_train)

# Predict on the validation set and compute the accuracy and precision scores
y_val_pred = lr_model.predict(X_val)

#Call to result_metrics
result_metrics(y_train, y_train_pred, y_val, y_val_pred, cv_results)

#save the model
save_model(lr_model, 'lr_model_cv')

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index, :-1]
        y = self.data[index, -1]
        return x, y

# Define the RNN model
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, hidden = self.rnn(x)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        return out


# Instantiate the dataset and dataloader
dataset = MyDataset(data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Instantiate the RNN model
model = MyRNN(input_size=train_data.shape[1] - 1, hidden_size=16, output_size=1)

# # Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")


In [ ]:
import numpy as np

num_samples = 1101000
seq_length = 11
num_features = 110

# assuming `data` is your input data of shape (1101000, 110)
data = np.random.rand(1101000, 110)

# truncate data to a multiple of (seq_length * num_features)
data = data[:num_samples * seq_length * num_features]

# reshape data into (num_samples, seq_length, num_features)
train_data_reshaped = data.reshape(1101000, 11, 10)

# convert data to a PyTorch tensor
data = torch.from_numpy(data).float()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split


class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hx=None, cx=None):
        if hx is None and cx is None:
            if len(x.shape) == 2:
                x = x.unsqueeze(2)  # add an extra dimension for the number of features
            batch_size, seq_len, num_features = x.shape
            hx = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
            cx = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
        else:
            batch_size = hx.shape[1]

        out, (hn, cn) = self.lstm(x, (hx, cx))
        out = out[:, -1, :].view(batch_size, -1)  # use only the last output of the sequence and reshape

        out = self.fc(out)

        return out


# Define the training function
def train_model(model, X_train, y_train, X_val, y_val, num_epochs=10, batch_size=128, learning_rate=0.001):
    train_losses = []
    val_losses = []
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_dataset = StockDataset(X_train, y_train)
    val_dataset = StockDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.argmax(dim=1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)
        model.eval()
        running_loss = 0.0
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(inputs)
                loss = criterion(outputs, labels.argmax(dim=1))
                running_loss += loss.item()
        val_loss = running_loss / len(val_loader)
        val_losses.append(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    return train_losses, val_losses

# Train multiple models and save them
num_models = 10
models = []
train_losses_list = []
val_losses_list = []
for i in range(num_models):
    model = LSTM(input_dim=108, hidden_dim=64, num_layers=2, output_dim=3).to(device)
    train_losses, val_losses = train_model(model, X_train, y_train, X_val, y_val)
    models.append(model)
    train_losses_list.append(train_losses)
    val_losses_list.append(val_losses)


TypeError: ignored

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load the training data
print(temp_data.shape, X.shape, y_encoded.shape)
train_data = temp_data

# Split the training data into a primary training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()

# Convert y_train and y_val to one-hot encoded vectors
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

# Check the shape of X_train and y_train
print(X_train.shape, y_train.shape)

# Define the neural network architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(100, train_data.shape[1]-1)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the primary training set
model.fit(X_train.reshape(-1, 100, train_data.shape[1]-1), y_train, batch_size=128, epochs=10, validation_data=(X_val.reshape(-1, 100, train_data.shape[1]-1), y_val))


(1100500, 109) (1100500, 108) (1100500, 3)


ValueError: ignored

In [ ]:
len(X_train)

880400